In [1]:
'''
A Convolutional Network implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

'\nA Convolutional Network implementation example using TensorFlow library.\nThis example is using the MNIST database of handwritten digits\n(http://yann.lecun.com/exdb/mnist/)\n\nAuthor: Aymeric Damien\nProject: https://github.com/aymericdamien/TensorFlow-Examples/\n'

In [2]:
import tensorflow as tf

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


모형을 구성하기 위한 파라미터들을 설정합니다.

* learning_rate: Gradient descent에서 사용할 학습율 파라미터
* training_iters: training을 몇바퀴 돌 것인지 설정
* batch_size: 한번의 배치에 몇개의 데이터를 받을지 결정
* display_step: 몇번의 step마다 중간결과를 출력할지 결정

* n_input: input의 차원수. 여기선 28x28 = 768
* n_classes: output의 차원수. 0~9까지 총 10개의 차원
* dropout:

In [3]:
# Parameters
learning_rate = 0.001
training_iters = 200000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

x, y에 placeholder를 지정하며 차원은 각각 n_input과 n_classes에 맞게 설정합니다.
keep_prob는 dropout probability를 지정할 수 있도록 placeholder를 만든 것입니다.
이는 test할때는 이 keep_prob를 1.0으로 수정하여 사용해야 하기 때문에 placeholder로 설정한 것입니다.

In [4]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

편의를 위해 여러 wrapper함수들을 정의합니다.

* conv2d
  * 2D convolution 연산을 도와주는 wrapper이며 tf.nn.conv2d와 bias_add, relu로 구성됩니다.
  * $ \text{relu}( \text{conv2d}(x, W) + b ) $
  * tensorflow에서 2d 는 기본적으로 NHWC (Num_sample x Height x Width x Channels) 구조를 따릅니다.
  * strides 옵션에서 0, 3차원이 1, 1인건 sample과 channel방향에 대해서는 convolution을 겹쳐서 하진 않겠다는 이야기입니다.
  * strides 값은 몇칸씩 filter(kernel)을 미끄러트리면서 Convolution을 수행할지를 의미합니다.
  * padding은 input의 주위 공간에 얼마나 zero padding을 할지를 결정해주는 값입니다.
      * 'SAME': input과 같은 크기로 나오도록 padding 결정, 'VALID' padding없이 정직하게 convolution 연산을 함. output 크기는 input에 비해 작아짐


In [5]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

* maxpool2d
    * 2차원 maxpooling 구성을 도와주기 위해 만든 wrapper 함수
    * ksize는 kernel size를 의미하고, 0번째, 3번째가 1인 이유는 위와 같이 num_sample과 channels 공간쪽으로는 pooling을 안하겠다는 이야기입니다.
    * strides는 미끄러지는 사이즈를 의미하고 보통의 풀링은 ksize와 strides를 동일하게 설정합니다. (그래야 overlap 되는 형태로 풀링이 되지 않습니다.)
    * padding은 conv2d와 거의 동일한 의미로 정의됩니다. 즉, SAME은 입/출력 크기가 같도록 zero padding이 되고, VALID는 줄어듭니다 차원이

In [6]:
def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

* conv_net 함수에서는 실제 모형을 구성하는 부분이 구현되어 있습니다.
* 받는 인자들
    * x: 데이터 input. shape=[None, 784]
    * weights: tf.Variable로 만들어진 weight 들을 저장하고 있는 dictionary 객체
    * biases: tf.Variable로 만들어진 bias 들을 저장하고 있는 dictionary 객체
    * dropout: dropout의 정도를 0.0~1.0으로 정의해 주는 변수를 넘김
* 내부 구현의 역할들 설명
    * tf.reshape를 통해 [None, 784] 형태의 데이터를 [None, 28, 28, 1] 형태로 바꾼다.
        * 이값의 순서는 위에서 설명한 NHWC에 따른 형태로 데이터를 변환할 것이다. 여기선 흑백 데이터이므로 Channel = 1이고, -1이 의미하는 바는 다른 차원들을 다 계산한 후 자동으로 -1 부분의 값을 채워 넣으라는 의미이다.
    * conv1, conv2는 위에서 정의한 conv2d와 maxpool2d를 이용해 convolution layer와 pooling layer를 정의한다.
        * 우측항을 통해 tf graph 내에서는 정상적으로 연결작업이 일어나므로 다른 task에 대해 변수를 중복적으로 사용해도 된다. (conv1을 conv2d와 maxpool2d 모두에 사용)
    * fcl은 실제 classifier로 연결되는 fully connected layer 구성하는 부분을 의미한다.
        * weights[wd1].get_shape().as_list()[0]은 wd1 weight의 input 차원을 의미이므로, [-1, ...]은 첫번째 차원은 batch_size 만큼의 인스턴스 수를 의미하고, 두번째 차원은 feature들을 한줄롤 쭉 나열한 것을 의미한다.
        * 위 과정을 통해 weights['wdl']와 곱해질 수 있는 준비를 마치고 tf.matmul을 통해 weight를 곱하고 Ax+b 를 위해 biases['bd1']을 더한다.
        * 이후 tf.nn.relu(fcl)을 통해 activation function을 붙인다.
        * fcl 마지막 부분에 tf.nn.dropout을 통해 dropout을 수행한다.
    * out은 최종 output layer로 역시 평범(?) 하게 Ax+b형태를 구성하고 이 결과값을 return 한다.

In [7]:
# Create model
def conv_net(x, weights, biases, dropout):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

위에서 작성한 함수들을 바탕으로 실제 모형을 구성하는 부분입니다.
* weights
    * wc1, wc2는 conv2d를 위한 filter로 [filter_height, filter_width, in_channels, out_channels] 형태를 따릅니다.
    * wdl이 7*7*64가 input 차원인 이유는 maxpool2d에서 'SAME'으로 2x2필터를 사용하였기 때문에 height, weight모두 한번 거칠때마다 1/2씩 준다는 걸 의미하므로 28 -> 14 -> 7 로 차원이 줄어든 결과임을 의미합니다.
    * wdl에서 두번째 차원 숫자인 1024는 fcl에서 사용할 hidden node의 숫자입니다.
    * out은 wdl의 1024개의 노드로부터 n_classes(=10)의 output을 생성하도록 weight를 설정한 것입니다.
* biases
    * bias들에 대해 tf.Variable로 정의합니다 기본적으로 위의 wc1, wc2, bdl, out의 output 차원과 차원크기를 공유합니다.
* `pred=conv_net()`을 통해 위의 함수를 기반으로 모형을 생성합니다.
* cost 부분은 `tf.nn.softmax_cross_entropy_with_logits()` 함수를 사용해 softmax loss를 한다음, tf.reduce_mean()으로 batch안에서의  loss의 평균값을 구합니다.
* optimizer는 위의 loss function을 tf.train.AdamOptimizer 를 통해 minimize하도록 설정합니다.
* correct_pred와 accuracy는 accuracy 수치를 구하기 위한 연산을 설정해 둔 것입니다.

In [8]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

MNIST 데이터셋을 통해 학습하도록 합니다.
주의하여 보아야 할 점은 학습시에는 keep_prob를 dropout 변수를 통해 1.0 미만의 값을 사용하지만 predict를 수행할때는 1.0을 사용한다는 점입니다.

In [9]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       keep_prob: dropout})
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc)
        step += 1
    print "Optimization Finished!"

    # Calculate accuracy for 256 mnist test images
    print "Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                      y: mnist.test.labels[:256],
                                      keep_prob: 1.})

Iter 1280, Minibatch Loss= 16965.662109, Training Accuracy= 0.29688
Iter 2560, Minibatch Loss= 6193.611328, Training Accuracy= 0.63281
Iter 3840, Minibatch Loss= 6702.179199, Training Accuracy= 0.68750
Iter 5120, Minibatch Loss= 4156.445312, Training Accuracy= 0.77344
Iter 6400, Minibatch Loss= 2642.371826, Training Accuracy= 0.83594
Iter 7680, Minibatch Loss= 3679.105713, Training Accuracy= 0.80469
Iter 8960, Minibatch Loss= 3457.911377, Training Accuracy= 0.79688
Iter 10240, Minibatch Loss= 2444.788818, Training Accuracy= 0.85938
Iter 11520, Minibatch Loss= 2988.898926, Training Accuracy= 0.82031
Iter 12800, Minibatch Loss= 1730.938354, Training Accuracy= 0.89844
Iter 14080, Minibatch Loss= 2937.672119, Training Accuracy= 0.88281
Iter 15360, Minibatch Loss= 2277.700195, Training Accuracy= 0.89844
Iter 16640, Minibatch Loss= 954.914246, Training Accuracy= 0.92188
Iter 17920, Minibatch Loss= 2524.730225, Training Accuracy= 0.85156
Iter 19200, Minibatch Loss= 1554.467285, Training Accur